In [1]:
import pandas as pd
import os
import pickle

In [2]:
cache_file = '../cache/bitcoin/data.pickle'
dataset_path = '../datasets/original/Bitcoin Historical Data(2010-2022).csv'

data = None

if os.path.isfile(cache_file):
    with open(cache_file, 'rb') as f:
        data = pickle.load(f)
else:
    data = pd.read_csv(
        filepath_or_buffer=dataset_path,
    )

    data.rename(columns=str.lower, inplace=True)

    data['date'] = pd.to_datetime(data['date']).dt.date

    data = data.sort_values(by='date').reset_index(drop=True)

    close = data.pop('close')

    data['close'] = close

    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)

data

,date,open,high,low,volume,close
0,2010-07-18,0.0,0.1,0.1,75,0.1
1,2010-07-19,0.1,0.1,0.1,574,0.1
2,2010-07-20,0.1,0.1,0.1,262,0.1
3,2010-07-21,0.1,0.1,0.1,575,0.1
4,2010-07-22,0.1,0.1,0.1,2160,0.1
...,...,...,...,...,...,...
4301,2022-04-27,38113.0,39447.0,37869.0,426830240,39243.0
4302,2022-04-28,39243.0,40382.0,38888.0,542391872,39748.0
4303,2022-04-29,39748.0,39919.0,38175.0,382904352,38581.0
4304,2022-04-30,38590.0,38776.0,37596.0,248220336,37650.0


In [3]:
data = data.dropna().reset_index(drop=True)

data.isna().sum()

date      0
open      0
high      0
low       0
volume    0
close     0
dtype: int64

In [4]:
dataset_path = '../datasets/consolidated/posts.csv'

data_posts = pd.read_csv(
    filepath_or_buffer=dataset_path,
    lineterminator='\n',
)

data_posts['date'] = pd.to_datetime(data_posts['date']).dt.date

data_posts

,date,sentiment
0,2013-01-21,0.296672
1,2013-01-22,0.357815
2,2013-01-23,0.391366
3,2013-01-24,0.196592
4,2013-01-25,0.259927
5,2013-01-26,0.160243
6,2013-01-27,0.265702
7,2013-01-28,0.269674
8,2013-01-29,0.229464
9,2013-01-30,0.230275


In [5]:
from datetime import timedelta

min_date = data_posts['date'].min()
max_date = data_posts['date'].max()

data = data[data['date'] >= min_date]
data = data[data['date'] <= max_date + timedelta(days=3)]

data = data.reset_index(drop=True)

data

,date,open,high,low,volume,close
0,2013-01-21,15.7,17.0,15.6,61502,16.8
1,2013-01-22,16.8,17.6,16.6,60975,17.3
2,2013-01-23,17.3,17.6,16.8,49439,17.5
3,2013-01-24,17.5,19.2,15.6,172009,16.9
4,2013-01-25,16.9,17.8,15.4,80767,17.4
5,2013-01-26,17.4,17.9,16.5,34811,17.9
6,2013-01-27,17.9,18.0,17.2,23767,17.8
7,2013-01-28,17.8,18.9,17.8,57669,18.7
8,2013-01-29,18.7,19.8,18.7,86082,19.5
9,2013-01-30,19.5,19.8,19.1,45065,19.7


In [6]:
data.to_csv('../datasets/consolidated/bitcoin.csv', index=False)